In [1]:
#IMPORTING the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display as ipd
from glob import glob
import librosa
import seaborn as sns
import librosa.display
import skimage.io
import os

In [2]:
import numpy as np
# Multihot to eventroll function
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.delete(active_events, 8)


In [3]:
# Making the numpy array of X input
X=np.zeros((10000,64,1000))
i=0
dir_name="../input/assigement2-data/X"
for filename in os.listdir("../input/assigement2-data/X"):
    #print(filename)
    file_path="../input/assigement2-data/X"+filename
    file=np.load(os.path.join(dir_name, filename))
    i=i+1
    sz=len(filename)
    j=0
    num=0
    while(filename[j]!='_'):
      j=j+1
    j=j+1
    while(j<sz-4):
      num=num*10+int(ord(filename[j])-ord('0'))
      j=j+1
    X[num]=file[0]
 

In [ ]:
print(X.shape)

In [4]:
# Making the Y input 
Y=np.zeros((10000,10))
i=0
dir_name2="../input/assigement2-data/Y"
for filename in os.listdir("../input/assigement2-data/Y"):
    # print(filename)
    file_path="../input/assigement2-data/Y"+filename
    file=np.load(os.path.join(dir_name2, filename))
    # labels.append(eventroll_to_multihot_vector(file))
    i=i+1
    sz=len(filename)
    j=0
    num=0
    while(filename[j]!='_'):
      j=j+1
    j=j+1
    while(j<sz-4):
      num=num*10+int(ord(filename[j])-ord('0'))
      j=j+1
    Y[num]=eventroll_to_multihot_vector(file)


In [ ]:
print(Y[0].shape)

In [5]:
# Counting the Specific class for checking imbalance
count=0
for i in range(1000):
    flag=0;
    for j in range(10):
        if(Y[i][j]==1 and j==8):
            flag=1
    if(flag):
        count=count+1
print(count)  

923


In [7]:
# for i in (list_y):
#   print(i)

In [9]:
# import librosa
# import librosa.display
# S_db = librosa.amplitude_to_db(arr_x[0], ref=np.max)
# plt.figure()
# librosa.display.specshow(S_db)
# plt.colorbar()

In [10]:
# Making the train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=100)

In [11]:
# Reshaping the data 
X_train = X_train.reshape(8000,64, 1000, 1)
X_test = X_test.reshape(2000,64, 1000, 1)
X_train = X_train.astype('float32')
X_test= X_test.astype('float32')

In [ ]:
print(X_train.shape)

In [12]:
import keras.backend as K
import tensorflow as tf
def get_f1(y_true, y_hat): #taken from old keras source code
    f1s = [0, 0, 0]

    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_hat, tf.float64)

    for i, axis in enumerate([None, 0]):
        TP = tf.count_nonzero(y_pred * y_true, axis=axis)
        FP = tf.count_nonzero(y_pred * (y_true - 1), axis=axis)
        FN = tf.count_nonzero((y_pred - 1) * y_true, axis=axis)

        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1 = 2 * precision * recall / (precision + recall)

        f1s[i] = tf.reduce_mean(f1)

    weights = tf.reduce_sum(y_true, axis=0)
    weights /= tf.reduce_sum(weights)

    f1s[2] = tf.reduce_sum(f1 * weights)

    micro, macro, weighted = f1s
    return micro, macro, weighted

In [13]:
# Library for focal loss
! pip install focal_loss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 63.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 69.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.5 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1
ERROR: pip's dependency resolver does not currently take into account

In [14]:
#Making the CNN Based Model
import keras
from keras import regularizers, activations
from tensorflow.keras import datasets, layers, models
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import layers
import tensorflow as tf
from focal_loss import BinaryFocalLoss


model = models.Sequential()
model.add(layers.Conv2D(24, (3, 3), activation='relu', input_shape=(64, 1000, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(48, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(GlobalAveragePooling2D())
model.add(layers.Dense(300, activation="relu"))
model.add(Dropout(0.2))
model.add(layers.Dense(200, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(10, activation='sigmoid'))

model.compile(loss=BinaryFocalLoss(gamma=2), metrics=[keras.metrics.BinaryAccuracy()], optimizer='adam')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 998, 24)       240       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 499, 24)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 497, 32)       6944      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 248, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 246, 48)       13872     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 123, 48)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 121, 64)        2

In [ ]:
from datetime import datetime 
num_epochs = 200
num_batch_size = 4
start = datetime.now()
history = model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), verbose=1)
duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
#Saving the  model
model.save("assigement2_f3.h5")

In [ ]:
# Plotting the model
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
y_hat=model.predict(X_test)

In [ ]:
y_hat=y_hat.round()

In [ ]:
lst=[0,0,0,0,0,0,0,0,0,0]
for i in range(400):
  for j in range(10):
    print(y_hat[i][j])
    print(y_test[i][j])
    if(y_hat[i][j]==y_test[i][j]):
       lst[j]=lst[j]+1
for j in lst:
  print(j/400)

In [ ]:
for j in lst:
  print(j/400)

In [ ]:
lst90=[0,0,0,0,0,0,0,0,0,0]
lst91=[0,0,0,0,0,0,0,0,0,0]
for i in range(400):
  for j in range(10):
    lst90[j]=lst90[j]+y_hat[i][j]
    lst91[j]=lst91[j]+y_test[i][j]


In [ ]:
print(lst90)

In [ ]:
lst91

In [ ]:
y_hat[6]

In [ ]:
count=0
for i in range(0,400):
  flag=1
  for j in range(0,10):
    if(y_hat[0][j] != y_hat[i][j]):
      flag=0
  count=count+flag
print(count)

# ***Validation step***

In [ ]:
val_x={}
list_valx=[]
i=0
dir_name2="../input/validation/dataset/X"
for filename in os.listdir("../input/validation/dataset/X"):
    #print(filename)
    file_path="../input/validation/dataset/X"+filename
    file=np.load(os.path.join(dir_name2, filename))
    i=i+1
    sz=len(filename)
    j=0
    num=0
    while(filename[j]!='_'):
      j=j+1
    j=j+1
    while(j<sz-4):
      num=num*10+int(ord(filename[j])-ord('0'))
      j=j+1
    val_x[num]=file[0]
    list_valx.append(num)
    #print(num)
 
print(len(val_x))


In [ ]:
print(len(list_valx))

In [ ]:
val_y={}
i=0
list_valy=[]
dir_name3="../input/validation/dataset/Y"
for filename in os.listdir("../input/validation/dataset/Y"):
    # print(filename)
    file_path="../input/validation/dataset/Y"+filename
    file=np.load(os.path.join(dir_name3, filename))
    # labels.append(eventroll_to_multihot_vector(file))
    i=i+1
    sz=len(filename)
    j=0
    num=0
    while(filename[j]!='_'):
      j=j+1
    j=j+1
    while(j<sz-4):
      num=num*10+int(ord(filename[j])-ord('0'))
      j=j+1
    val_y[num]=eventroll_to_multihot_vector(file)
    list_valy.append(num)


In [ ]:
print(len(val_y))

In [ ]:
x_val=[]
y_val=[]
for i in list_valx:
  x_val.append(val_x[i])
  y_val.append(val_y[i])

In [ ]:
print(len(x_val))

In [ ]:
x_val=np.array(x_val)
x_val = x_val.reshape(2000,64, 1000, 1)


In [ ]:
import keras
from focal_loss import BinaryFocalLoss

In [ ]:
model =  keras.models.load_model("../input/parametrs/assigement2_f.h5")

In [ ]:
val_pred = model.predict(x_val)

In [ ]:
val_pred=val_pred.round()

In [ ]:
lst000=[0,0,0,0,0,0,0,0,0,0]
lst001=[0,0,0,0,0,0,0,0,0,0]
for i in range(2000):
  for j in range(10):
    lst000[j]=lst000[j]+val_pred[i][j]
    lst001[j]=lst001[j]+y_val[i][j]

In [ ]:
lst000

In [ ]:
lst001

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print(precision_score(val_pred,y_val,average="samples"))
print(recall_score(val_pred,y_val,average="samples"))
print(f1_score(val_pred,y_val,average="samples"))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_val, val_pred)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_val, val_pred))

In [ ]:
lst12=[0,0,0,0,0,0,0,0,0,0]
count=0
for i in range(2000):
  for j in range(10):
    if(val_pred[i][j]==y_val[i][j]):
       lst12[j]=lst12[j]+1

for i in range(2000):
  flag=1
  for j in range(10):
    if(val_pred[i][j]!=y_val[i][j]):
       flag=0
  count=count+flag

for j in lst12:
  print(j/2000)
print("final ccc")
print(count/2000)

In [ ]:
! pip install focal_loss

In [ ]:
import tensorflow as tf
import keras
from keras import regularizers, activations
from tensorflow.keras import datasets, layers, models
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import layers
import tensorflow as tf
from focal_loss import BinaryFocalLoss
model =  keras.models.load_model("../input/parametrs/assigement2_f.h5")

In [ ]:
val_pred1 = model.predict(x_val)

In [ ]:
val_pred1=val_pred1.round()

In [ ]:
lst000=[0,0,0,0,0,0,0,0,0,0]
lst001=[0,0,0,0,0,0,0,0,0,0]
for i in range(2000):
  for j in range(10):
    lst000[j]=lst000[j]+val_pred1[i][j]
    lst001[j]=lst001[j]+y_val[i][j]

In [ ]:
lst000

In [ ]:
lst001

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print(precision_score(val_pred1,y_val,average="samples"))
print(recall_score(val_pred1,y_val,average="samples"))
print(f1_score(val_pred1,y_val,average="samples"))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_val, val_pred1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, val_pred1))

In [ ]:
lst13=[0,0,0,0,0,0,0,0,0,0]
count=0
for i in range(2000):
  for j in range(10):
    if(val_pred1[i][j]==y_val[i][j]):
       lst13[j]=lst13[j]+1

for i in range(2000):
  flag=1
  for j in range(10):
    if(val_pred1[i][j]!=y_val[i][j]):
       flag=0
  count=count+flag

for j in lst13:
  print(j/2000)
print("final ccc")
print(count/2000)

# # ***Making Prediction***

In [ ]:
! pip install focal_loss

In [ ]:
import tensorflow as tf
import keras
from keras import regularizers, activations
from tensorflow.keras import datasets, layers, models
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import layers
import tensorflow as tf
from focal_loss import BinaryFocalLoss
model =  keras.models.load_model("../input/parametrs/assigement2_f.h5")

In [ ]:
test_x={}
list_testx=[]
fname=[]
i=0
dir_name2="../input/ass2-test-data/test/X"
for filename in os.listdir("../input/ass2-test-data/test/X"):
    #print(filename)
    file_path="../input/ass2-test-data/test/X"+filename
    file=np.load(os.path.join(dir_name2, filename))
    i=i+1
    sz=len(filename)
    j=0
    num=0
    while(filename[j]!='_'):
      j=j+1
    j=j+1
    while(j<sz-4):
      num=num*10+int(ord(filename[j])-ord('0'))
      j=j+1
    test_x[num]=file[0]
    list_testx.append(num)
    fname.append(filename)
    #print(num)
 
print(len(test_x))


In [ ]:
test_y={}
i=0
list_valy=[]
dir_name3="../input/groud-truth/lables_test"
for filename in os.listdir("../input/groud-truth/lables_test"):
    # print(filename)
    file_path="../input/groud-truth/lables_test"+filename
    file=np.load(os.path.join(dir_name3, filename))
    # labels.append(eventroll_to_multihot_vector(file))
    i=i+1
    sz=len(filename)
    j=0
    num=0
    while(filename[j]!='_'):
      j=j+1
    j=j+1
    while(j<sz-4):
      num=num*10+int(ord(filename[j])-ord('0'))
      j=j+1
    test_y[num]=eventroll_to_multihot_vector(file)
    list_valy.append(num)

In [ ]:
x_test=[]
y_test=[]
for i in list_testx:
    x_test.append(test_x[i])
    y_test.append(test_y[i])
  
  

In [ ]:
print(len(y_test))

In [ ]:
y_test=np.array(y_test)

In [ ]:
y_test.shape

In [ ]:
x_test=np.array(x_test)
x_test = x_test.reshape(2500,64, 1000, 1)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
print(predictions.shape)

In [ ]:
predictions=predictions.round()

In [ ]:
for i in predictions:
    print(i)

In [ ]:
lst100=[0,0,0,0,0,0,0,0,0,0]
lst101=[0,0,0,0,0,0,0,0,0,0]
for i in range(2500):
  for j in range(10):
    lst100[j]=lst100[j]+predictions[i][j]
    lst101[j]=lst101[j]+y_test[i][j]

In [ ]:
lst100

In [ ]:
lst101

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print(precision_score(predictions,y_test,average="samples"))
print(recall_score(predictions,y_test,average="samples"))
print(f1_score(predictions,y_test,average="samples"))

In [ ]:
from numpy import argmax
preds=[]
annotate = ['Alarm_bell_ringing','Blender','Cat','Dishes','Dog','Electric_shaver_toothbrush','Frying','Running_water','Speech','Vacuum_cleaner']
for p in predictions:
    one = []
    for i in range(0,10):
        if p[i]==1:
            one.append(annotate[i])
    if(len(one)==0):
        one.append('Silence')
    one = np.array(one)
    preds.append(one)


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test, predictions)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

In [ ]:
lst13=[0,0,0,0,0,0,0,0,0,0]
count=0
for i in range(2000):
  for j in range(10):
    if(predictions[i][j]==y_test[i][j]):
       lst13[j]=lst13[j]+1

for i in range(2000):
  flag=1
  for j in range(10):
    if(predictions[i][j]!=y_test[i][j]):
       flag=0
  count=count+flag

for j in lst13:
  print(j/2000)
print("final ccc")
print(count/2000)

In [ ]:
for i in preds:
    print(i)

In [ ]:
df1 = pd.DataFrame({"fname" : fname, "predictions" : preds})
df1.to_csv("ass2_submission_f.csv", index=False)